<center><p style = "background-color : gold; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Подгружаем библиотеки</span>

In [6]:
import requests # HTTP-запросов к веб-страницам (получение HTML-контента)
import time # работа с временными задержками (паузы между запросами)
import pandas as pd # ну тут уж ясно таблички наши
from bs4 import BeautifulSoup # парсинг HTML/XML (извлечение данных из сырого HTML)
from selenium import webdriver # модуль для автоматизации браузера (управление Chrome)
from selenium.webdriver.chrome.options import Options # настройка опций Chrome (например, headless-режим)
from selenium.webdriver.chrome.service import Service # управление сервисом ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager # автоматическая загрузки и управления версиями ChromeDriver
from selenium.webdriver.common.by import By # поиск элементов на странице по различным критериям (CSS-селекторы, XPath)
from urllib.parse import urljoin # поиск элементов на странице по различным критериям (CSS-селекторы, XPath)
from tqdm import tqdm  # отображения прогресса, чтоб было ясно, что не просто так крутим колесико

<center><p style = "background-color : gold; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Автоматизируем сбор структурированных данных о стратегиях для последующего анализа, сравнения и принятия решений в трейдинге.</span>

In [7]:
# Настройка драйвера
chrome_options = Options()  # Создаем объект для настройки Chrome
chrome_options.add_argument("--headless")  # Активируем headless-режим
driver = webdriver.Chrome(  # Инициализируем драйвер
    service=Service(  # Настраиваем сервис для ChromeDriver
        ChromeDriverManager().install()  # Автоматическая установка драйвера
    ),
    options=chrome_options  # Применяем настройки браузера
)

# Сбор всех страниц
all_title = []  # Создаем список для хранения заголовков стратегий
all_links = []  # Создаем список для хранения ссылок на стратегии

<center><p style = "background-color : gold; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Создаем цикл обработки страниц страницы с 1 по 42</span>

In [8]:
# Создаем цикл обработки страниц 
for page in tqdm(range(1,43), desc="Сбор страниц"):
    # Делаем динамическую подстановку страниц в наш код:
    if page == 1:
        url = "https://www.tradingview.com/scripts/?script_type=strategies"
    else:
        url = f"https://www.tradingview.com/scripts/page-{page}/?script_type=strategies"
    
    # Выполняем обработку HTTP-запросов
    try:
        response = requests.get(url, timeout=10) # Отправляем get-запрос странице
        response.raise_for_status() # Проверяем код ответа на исключения (ошибки 404 и т.п.)
        # Парсим и ищем элементы по CSS-селектору
        soup = BeautifulSoup(response.content, "html.parser")
        data = soup.select('a[class="title-tkslJwxl line-clamp-tkslJwxl stretched-outline-tkslJwxl"]')
        # Собираем данные
        for item in data:
            all_title.append(item.text) # Извлекем текст из ссылки
            all_links.append(urljoin(url, item['href'])) # Преобразуем ссылки из относительных в абсолютные
        
        time.sleep(1)  # Задержка между запросами (Шоб не заблочило чего вдруг)
    
    # Обработка ошибок 
    except Exception as e:
        print(f"Ошибка при обработке страницы {page}: {str(e)}")

Сбор страниц: 100%|██████████| 42/42 [01:01<00:00,  1.47s/it]


<center><p style = "background-color : gold; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Создаем фукнцию обработки финансовых данных со страниц стратегий.</span>

In [9]:
"""def parse_overview(total_p_l_element, financial_elements):

    financial_list = []
    # Основная логика парсинга
    try:
        # Проверка существования элементов, извлечение
        financial_list.append(total_p_l_element[0].get_attribute('innerHTML') if total_p_l_element else '')
        # Цикл по дополнительным метрикам(Просадка, кол-во сделаок, прибыль, профит) 
        for element in financial_elements:
            financial_list.append(element.get_attribute('innerHTML'))
    # Обработка ошибок
    except Exception as e:
        print(f"Ошибка при парсинге данных вкладки overview: {str(e)}")
        financial_list.extend(['']*5)
    
    return financial_list[:5]"""

def parse_tags(tags_element):
    
    tags = set()
    
    try:
        for elem in tags_element:
            tags.add(elem.get_attribute('innerHTML'))
    except Exception as e:
        print(f"Ошибка при парсинге тегов: {str(e)}")
    
    return tags

def parse_performance(performance_elements_title, link):
    
    performance_data = {}
    i = 1 

    try:
        for title in performance_elements_title:
            performance_data[title.get_attribute('textContent')] = driver.find_element(By.XPATH,
                                                                                    f'//table/tbody/tr[{i}]/td[2]/div/div[1]/div[1]').text
            i+=1
    except Exception as e:
        print(f"Ошибка при парсинге данных вкладки performance: {str(e)}, при обработке ссылки {link}")
    
    return performance_data

def parse_trade_analysis(trade_analysis_elements_title, link):

    trade_analysis_data = {}
    i = 1 

    try:
        for title in trade_analysis_elements_title:
            trade_analysis_data[title.get_attribute('textContent')] = driver.find_element(By.XPATH,
                                                                                        f'//table/tbody/tr[{i}]/td[2]/div/div[1]/div[1]').text
            i+=1
    except Exception as e:
        print(f"Ошибка при парсинге данных вкладки trade_analysis: {str(e)}, при обработке ссылки {link}")
    
    return trade_analysis_data

def parse_ratios(ratios_elements_title, link):
    
    ratios = {}
    i = 1 

    try:
        for title in ratios_elements_title:
            ratios[title.get_attribute('textContent')] = driver.find_element(By.XPATH,
                                                                            f'//table/tbody/tr[{i}]/td[2]/div/div[1]/div[1]').text
            i+=1
    except Exception as e:
        print(f"Ошибка при парсинге данных вкладки ratios: {str(e)}, при обработке ссылки {link}")
    
    return ratios

def parse_properties(properties_elements_title, properties_elements_data, link):
    
    properties = {}
    
    try:
        for title, data in zip(properties_elements_title, properties_elements_data):
            properties[title.get_attribute('textContent')] = data.get_attribute('textContent')
    except Exception as e:
        print(f"Ошибка при парсинге данных вкладки properties: {str(e)}, при обработке ссылки {link}")
    
    return properties

<center><p style = "background-color : #40e0d0; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Непосредственный сбор:</span>

In [10]:
# Парсинг данных с индивидуальных страниц стратегий
strategy_description = [] 
tags = [] 
#overview_data = [] 
performance_data = []
trade_analysis_data=[]
ratios_data = []
properties_data = []

# Итерируемся по индивидуальным ссылкам стратегий

for link in tqdm(all_links, desc='Парсинг данных с индивидуальных страниц стратегий'):
    try:

        driver.get(link) # Переход на страницу конкретной стратегии
        
        #Парсинг описания и тегов стратегии

        try:
            description_element = driver.find_elements(By.CLASS_NAME, "ast-_PPhhqv6")
            tags_element = driver.find_elements(By.CLASS_NAME, "xsmall-rVj4hiuX")

            strategy_description.append(description_element[0].get_attribute('textContent'))
            tags.append(parse_tags(tags_element))
            
        except Exception as e:
            print(f"Ошибка в {link}: {str(e)} при парсинге описания стратегии и тегов")
            strategy_description.append('')
            tags.append({})

        """# Парсинг данных вкладки Overview
        
        try:
            element = driver.find_element(By.XPATH, '//*[@id="strategy"]') 
            element.click()
            time.sleep(1.5)  
            
            total_p_l_element = driver.find_elements(By.CLASS_NAME, "highlightedValue-LVMgafTl")
            overview_elements = driver.find_elements(By.CLASS_NAME, "value-LVMgafTl")
            overview_data.append(parse_overview(total_p_l_element, overview_elements))
            
        #  Ошибки при работе со вкладкой Overview
        except Exception as e:
            print(f"Ошибка в {link}: {str(e)}")
            overview_data.append(['']*5)"""

        # Парсинг вкладки Performance
        
        try:
            
            element = driver.find_element(By.XPATH, '//*[@id="strategy"]') 
            element.click()
            time.sleep(5)  
            
            element = driver.find_element(By.XPATH, '//*[@id="Performance"]') 
            element.click()
            time.sleep(1.5)  

            performance_elements_title = driver.find_elements(By.CLASS_NAME, "apply-overflow-tooltip.title-NcOKy65p")

            performance_data.append(parse_performance(performance_elements_title, link))
            
        #  Ошибки при открытии вкладки Performance
        except Exception as e:
            print(f"Ошибка в {link}: {str(e)} при открытии вкладки Performance")
            performance_data.append({})

         # Парсинг вкладки Trade analysis
        
        try:
            
            element = driver.find_element(By.XPATH, '//*[@id="Trades Analysis"]') 
            element.click()
            time.sleep(1.5)  

            trade_analysis_elements_title = driver.find_elements(By.CLASS_NAME, "apply-overflow-tooltip.title-NcOKy65p")

            trade_analysis_data.append(parse_trade_analysis(trade_analysis_elements_title, link))
            
        #  Ошибки при открытии вкладки Trade analysis
        except Exception as e:
            print(f"Ошибка в {link}: {str(e)} при открытии вкладки Trade analysis")
            trade_analysis_data.append({})
        
        # Парсинг данных вкладки Risk/Performance
        
        try:
            element = driver.find_element(By.XPATH, '//*[@id="Ratios"]') 
            element.click()
            time.sleep(1.5)  
            
            # Сбор показателей вкладки Risk/Performance
            ratios_elements_title = driver.find_elements(By.CLASS_NAME, "apply-overflow-tooltip.title-NcOKy65p")
            ratios_data.append(parse_ratios(ratios_elements_title, link))
            
        #  Ошибки при открытии вкладки Risk/Performance
        except Exception as e:
            print(f"Ошибка в {link}: {str(e)}, при открытии вкладки Risk/Performance")
            ratios_data.append({})
        
        # Парсинг данных вкладки Properties
        
        try:
            #Переход на вкладку Properties
            element = driver.find_element(By.XPATH, '//*[@id="Properties"]') 
            element.click()
            time.sleep(1.5)

            #Раскрытие всех вложенных вкладок

            subpage = 1

            for i in range(1, 5):
                element = driver.find_element(By.XPATH, f'//*[@id="tv-content"]/div/div/div/div[1]/div[2]/section/div[2]/div[2]/div/div/div/div/div[3]/div[{i}]/button')
                element.click()
                time.sleep(1.5)
                subpage+=1

            properties_elements_title = driver.find_elements(By.CLASS_NAME, "elemTitle-_SEShh26")
            properties_elements_data = driver.find_elements(By.CLASS_NAME, "elemData-_SEShh26")
            properties_data.append(parse_properties(properties_elements_title, properties_elements_data, link))
            
        #  Ошибки при открытии вкладки (подвкладок) Properties
        except Exception as e:
            print(f"Ошибка в {link}: {str(e)}, при открытии подвкладки {subpage} Properties")
            if subpage == 1:
                properties_data.append({})
            else:
                properties_elements_title = driver.find_elements(By.CLASS_NAME, "elemTitle-_SEShh26")
                properties_elements_data = driver.find_elements(By.CLASS_NAME, "elemData-_SEShh26")
                properties_data.append(parse_properties(properties_elements_title, properties_elements_data, link))

    # Обработка критических ошибок (Глобальные ошибки (например, недоступность страницы))
    except Exception as e:
        print(f"Критическая ошибка при обработке {link}: {str(e)}")
        strategy_description.append('')
        tags.append({})
        performance_data.append({})
        trade_analysis_data.append({})
        ratios_data.append({})
        properties_data.append({})
        continue

Парсинг данных с индивидуальных страниц стратегий:   0%|          | 3/959 [00:59<5:14:36, 19.75s/it]

Ошибка при парсинге данных вкладки trade_analysis: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table/tbody/tr[5]/td[2]/div/div[1]/div[1]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007FD363+60275]
	GetHandleVerifier [0x007FD3A4+60340]
	(No symbol) [0x006306F3]
	(No symbol) [0x00678690]
	(No symbol) [0x00678A2B]
	(No symbol) [0x006C0EE2]
	(No symbol) [0x0069D0D4]
	(No symbol) [0x006BE6EB]
	(No symbol) [0x0069CE86]
	(No symbol) [0x0066C623]
	(No symbol) [0x0066D474]
	GetHandleVerifier [0x00A48FE3+2467827]
	GetHandleVerifier [0x00A445E6+2448886]
	GetHandleVerifier [0x00A5F80C+2560028]
	GetHandleVerifier [0x00813DF5+153093]
	GetHandleVerifier [0x0081A3BD+179149]
	GetHandleVerifier [0x00804BB8+91080]
	GetHandleVerifier [0x00804D60+91504]
	GetHandleVerifier [0x007EFA10+4640]
	B

Парсинг данных с индивидуальных страниц стратегий:   1%|          | 5/959 [01:31<4:31:18, 17.06s/it]

Ошибка в https://www.tradingview.com/script/YorUiP1I-Guaranteed-Entry-Strategy-Every-5-Bars/: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="tv-content"]/div/div/div/div[1]/div[2]/section/div[2]/div[2]/div/div/div/div/div[3]/div[1]/button"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007FD363+60275]
	GetHandleVerifier [0x007FD3A4+60340]
	(No symbol) [0x006306F3]
	(No symbol) [0x00678690]
	(No symbol) [0x00678A2B]
	(No symbol) [0x006C0EE2]
	(No symbol) [0x0069D0D4]
	(No symbol) [0x006BE6EB]
	(No symbol) [0x0069CE86]
	(No symbol) [0x0066C623]
	(No symbol) [0x0066D474]
	GetHandleVerifier [0x00A48FE3+2467827]
	GetHandleVerifier [0x00A445E6+2448886]
	GetHandleVerifier [0x00A5F80C+2560028]
	GetHandleVerifier [0x00813DF5+153093]
	GetHandleVerifier [0x0081A3BD+179149]
	GetHandl

Парсинг данных с индивидуальных страниц стратегий:   1%|          | 9/959 [02:50<5:00:54, 19.00s/it]

...

Парсинг данных с индивидуальных страниц стратегий:  99%|█████████▊| 947/959 [4:55:06<03:46, 18.91s/it]

Ошибка при парсинге данных вкладки trade_analysis: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table/tbody/tr[5]/td[2]/div/div[1]/div[1]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007FD363+60275]
	GetHandleVerifier [0x007FD3A4+60340]
	(No symbol) [0x006306F3]
	(No symbol) [0x00678690]
	(No symbol) [0x00678A2B]
	(No symbol) [0x006C0EE2]
	(No symbol) [0x0069D0D4]
	(No symbol) [0x006BE6EB]
	(No symbol) [0x0069CE86]
	(No symbol) [0x0066C623]
	(No symbol) [0x0066D474]
	GetHandleVerifier [0x00A48FE3+2467827]
	GetHandleVerifier [0x00A445E6+2448886]
	GetHandleVerifier [0x00A5F80C+2560028]
	GetHandleVerifier [0x00813DF5+153093]
	GetHandleVerifier [0x0081A3BD+179149]
	GetHandleVerifier [0x00804BB8+91080]
	GetHandleVerifier [0x00804D60+91504]
	GetHandleVerifier [0x007EFA10+4640]
	B

Парсинг данных с индивидуальных страниц стратегий: 100%|█████████▉| 955/959 [4:57:38<01:16, 19.05s/it]

Ошибка при парсинге данных вкладки trade_analysis: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table/tbody/tr[8]/td[2]/div/div[1]/div[1]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007FD363+60275]
	GetHandleVerifier [0x007FD3A4+60340]
	(No symbol) [0x006306F3]
	(No symbol) [0x00678690]
	(No symbol) [0x00678A2B]
	(No symbol) [0x006C0EE2]
	(No symbol) [0x0069D0D4]
	(No symbol) [0x006BE6EB]
	(No symbol) [0x0069CE86]
	(No symbol) [0x0066C623]
	(No symbol) [0x0066D474]
	GetHandleVerifier [0x00A48FE3+2467827]
	GetHandleVerifier [0x00A445E6+2448886]
	GetHandleVerifier [0x00A5F80C+2560028]
	GetHandleVerifier [0x00813DF5+153093]
	GetHandleVerifier [0x0081A3BD+179149]
	GetHandleVerifier [0x00804BB8+91080]
	GetHandleVerifier [0x00804D60+91504]
	GetHandleVerifier [0x007EFA10+4640]
	B

Парсинг данных с индивидуальных страниц стратегий: 100%|██████████| 959/959 [4:58:54<00:00, 18.70s/it]


<center><p style = "background-color : white; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Датафрейм на выход:</span>

In [11]:
# Создание DataFrame
description_df = pd.DataFrame({'title': all_title, 'strategy_description': strategy_description, 'tags': tags})
performance_df = pd.DataFrame(performance_data)
trade_analysis_df = pd.DataFrame(trade_analysis_data)
ratios_df = pd.DataFrame(ratios_data)
#properties_df = pd.DataFrame(properties_data)

main_df = pd.concat([description_df, performance_df, trade_analysis_df, ratios_df], axis=1)
main_df['properties'] = properties_data

display(main_df)
display(main_df.info())

# Закрытие драйвера
driver.quit()

# Сохранение результатов
main_df.to_csv('tradingview_strategies_full.csv', index=False)

,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Largest losing trade,Largest losing trade percent,Avg # bars in trades,Avg # bars in winning trades,Avg # bars in losing trades,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties
0,Daily Breakout + Daily Shadow By Rouro,This script is a Pine v5 strategy designed to ...,"{statistics, Candlestick analysis, Chart patte...",0,"+8,375.00","9,973.00","1,598.00",0,"+117,304.17","8,375.00",...,"1,598.00",0.51%,50,63,1,0.203,0.453,6.241,0,"{'Trading range': 'Mar 10, 2025, 23:05 — May 0..."
1,Order Block Strategy,Strategy OverviewKey FeaturesOrder Block Detec...,"{Pivot points and levels, Pine utilities, Volume}","−1,453.09","+9,783.49","9,783.49",0.00001,0,−72.38,"10,667.89",...,0.000004,3.49%,"1,152","1,348",71,0.616,9.392,"1,426,164,099.778",0,"{'Trading range': 'Oct 01, 2024, 15:30 — May 0..."
2,EXODUS,EXODUS by (D...,"{Volume, Volatility, Oscillators}",0,"+14,293.23","68,836.40","54,543.18","1,672.77","+5,593,815.28","20,726.51",...,"1,767.68",4.71%,4,5,3,−1.136,−0.782,1.262,0,"{'Trading range': 'Jan 30, 2020, 11:00 — May 0..."
3,External Signals Strategy Tester v5,External Signals Strategy Tester v5 – User Gui...,"{educational, Bollinger Bands (BB), Bands and ...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Trading range': ' — ', 'Backtesting range': ..."
4,Guaranteed Entry Strategy (Every 5 Bars),"//version=5strategy(""Guaranteed Entry Strategy...",{regressions},NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,rt maax EMA cross strategy,this just sample of our strategies we publishe...,"{BTC, sma, trend, Trend Analysis, Forex, USD, ...","+39,115.55","+1,066,063.89","1,108,975.30","42,911.41","12,611.77","+9,960,471.92","1,106,625.50",...,"11,085.32",10.95%,766,952,282,0.024,2.104,25.843,0,"{'Trading range': 'Feb 01, 1933 — Oct 01, 2024..."
955,SPX Fair Value Strategy Ultimate,This is a strategy using the SPX Fair Value de...,{Oscillators},0,"+469,888.27","469,888.27",0,0,"−167,001.45","469,888.27",...,NaN,NaN,NaN,NaN,NaN,0.607,28.369,NaN,NaN,"{'Trading range': 'Jan 03, 2022 — Nov 22, 2022..."
956,Multi Trend Cross Strategy Template,Today I am sharing with the community trend cr...,"{regressions, strategy, Trend Analysis, Moving...",0,"+28,127,665.94","31,327,842.85","3,200,176.91","125,781.93","+13,648,918.30","28,133,021.71",...,"1,242,107.46",15.16%,151,262,40,0.137,1.885,9.789,0,"{'Trading range': 'Nov 03, 2010 — Oct 26, 2022..."
957,BOLLY Bands,This is a strategy using Bollinger Bands. The ...,"{bollingerbandstrategy, Bollinger Bands (BB), ...",0,"+7,998.34","11,559.50","3,561.16","1,448.00","+3,159.02","8,014.34",...,224.51,0.22%,57,51,72,1.367,443.519,3.246,0,"{'Trading range': 'Oct 17, 2022, 00:35 — Oct 2..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   title                          959 non-null    object
 1   strategy_description           959 non-null    object
 2   tags                           959 non-null    object
 3   Open P&L                       944 non-null    object
 4   Net profit                     944 non-null    object
 5   Gross profit                   944 non-null    object
 6   Gross loss                     944 non-null    object
 7   Commission paid                944 non-null    object
 8   Buy & hold return              944 non-null    object
 9   Max equity run-up              944 non-null    object
 10  Max equity drawdown            944 non-null    object
 11  Max contracts held             944 non-null    object
 12  Total trades                   944 non-null    object
 13  Total

None

<div class="alert alert-warning alert-dismissible">
 <strong>ВОПРОСЫ НА ПОДУМАТЬ:</strong>

Вот вкладки которые надо еще запарсить:
1. МЫ собрали всё
2. Производительность(Прибыль, доходы, приросты
3. Анализ сделок
4. Соотношения риска и доходности (ВАЖНАЯ) - можно интерпретировать её эффективность
5. Список сделок (ВАЖНАЯ) - определить популярность
6. Свойства(ВАЖНАЯ) - Тут информация по Типу графикаЮ Таймфреме, Валюте, и размеров тика + точность)


+ предложение собрать по стратегиям исходный код(это отдельный датафрейм) Можно попробововать пустить на анализ модельки этих кодов, посмотрим что она выдаст и какие даст по ним выводы. Как это реализовать ХЗ. Но думаю стоит отметить как идею

In [12]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   title                          959 non-null    object
 1   strategy_description           959 non-null    object
 2   tags                           959 non-null    object
 3   Open P&L                       944 non-null    object
 4   Net profit                     944 non-null    object
 5   Gross profit                   944 non-null    object
 6   Gross loss                     944 non-null    object
 7   Commission paid                944 non-null    object
 8   Buy & hold return              944 non-null    object
 9   Max equity run-up              944 non-null    object
 10  Max equity drawdown            944 non-null    object
 11  Max contracts held             944 non-null    object
 12  Total trades                   944 non-null    object
 13  Total